In [1]:
import folium
import h3
import pandas as pd

# Cargar los datos de los aeropuertos
airports = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat',
                       header=None, names=['airport_id', 'name', 'city', 'country', 'iata', 'icao', 'lat', 'lon',
                                           'alt', 'timezone', 'dst', 'tz', 'type', 'source'])

# Seleccionar solo los aeropuertos en los Estados Unidos
us_airports = airports[airports['country'] == 'United States']

# Crear una lista de hexágonos H3 que contienen los aeropuertos
hexagons = []
for i, row in us_airports.iterrows():
    lat, lon = row['lat'], row['lon']
    hex_id = h3.geo_to_h3(lat, lon, 6)  # 6 es el nivel de resolución de los hexágonos
    hexagons.append(hex_id)

# Contar la frecuencia de los hexágonos
hex_counts = pd.Series(hexagons).value_counts().reset_index()
hex_counts.columns = ['hex_id', 'count']

# Crear un mapa con folium y hexágonos H3
m = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

for i, row in hex_counts.iterrows():
    hex_id = row['hex_id']
    count = row['count']
    lat, lon = h3.h3_to_geo(hex_id)
    vertices = h3.h3_to_geo_boundary(hex_id)
    hex_coords = [[vertex[1], vertex[0]] for vertex in vertices]
    hex_color = 'Reds'  # Colores disponibles: https://github.com/dsc/colorbrewer-python

    folium.Polygon(
        locations=hex_coords,
        color=hex_color,
        fill=True,
        fill_color=hex_color,
        fill_opacity=0.7,
        tooltip=f"{count} aeropuertos"
    ).add_to(m)

# Mostrar el mapa
m


In [2]:
airports = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat',
                       header=None, names=['airport_id', 'name', 'city', 'country', 'iata', 'icao', 'lat', 'lon',
                                           'alt', 'timezone', 'dst', 'tz', 'type', 'source'])
airports.head()

,airport_id,name,city,country,iata,icao,lat,lon,alt,timezone,dst,tz,type,source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


In [3]:
data = airports.copy()

In [14]:
import folium
import h3
import numpy as np

# Crear un mapa centrado en una ubicación específica
mapa = folium.Map(location=[40.7128, -74.0060], zoom_start=10)

# Crear un conjunto de puntos de coordenadas aleatorias
puntos = [(40.7128 + 0.01*np.random.randn(), -74.0060 + 0.01*np.random.randn()) for _ in range(100)]

# Agregar una capa de hexágonos H3 al mapa
resolucion = 7 # la resolución de los hexágonos H3
hexagonos = []
for punto in puntos:
    indice_hex = h3.geo_to_h3(punto[0], punto[1], resolucion)
    coordenadas_hex = h3.h3_to_geo_boundary(indice_hex)
    hexagonos.append(coordenadas_hex)
    folium.Polygon(locations=coordenadas_hex, color='blue', fill=True, fill_opacity=0.5).add_to(mapa)

for punto in puntos:
    folium.Marker(location=punto, popup=str(punto)).add_to(mapa)

# Mostrar el mapa
mapa
